### 📦 Importação de Bibliotecas

Este bloco realiza a importação das bibliotecas e módulos necessários para o processamento, análise e visualização de grafos com dados em formato `.csv`.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import csv
import os
from networkx.algorithms.community import louvain_communities, girvan_newman
from itertools import islice 
import networkx as nx
from collections import Counter

### Definição de Variáveis

- **`input_path`**: Caminho do arquivo de entrada compactado `.txt.gz` contendo os dados originais. Este arquivo é o ponto de partida para a conversão.
  - **Exemplo**: `../data/twitter_combined.txt.gz`
  
- **`output_path`**: Caminho para o arquivo `.csv` gerado a partir do arquivo original. Este arquivo armazenará as arestas do grafo extraídas do arquivo compactado.
  - **Exemplo**: `../csv_files/twitter_network.csv`
  
- **`CSV_FILE`**: Caminho para o arquivo `.csv` **completo**, utilizado nas execuções finais do processo. Esse arquivo contém todos os dados extraídos e processados do arquivo original.
  
- **`CSV_FILE_SAMPLE`**: Caminho para o arquivo `.csv` **reduzido**, utilizado para desenvolvimento rápido e testes com amostras do conjunto de dados. Esse arquivo contém apenas uma fração dos dados, facilitando o desenvolvimento e o teste de funcionalidades.


In [ ]:
#Definiçao de variáveis
input_path = "../data/twitter_combined.txt.gz"
output_path = "../csv_files/twitter_network.csv"

CSV_FILE = '../csv_files/twitter_network.csv' # Arquivo completo
CSV_FILE_SAMPLE = '../csv_files/twitter_network_sample.csv' # Amostra para desenvolvimento rápido

### Definição de Constantes de Execução

- **`USE_SAMPLE = True` ou `False`**: Esta constante define se os algoritmos serão executados utilizando uma **amostra** do conjunto de dados ou o **arquivo completo**.
  - **`True`**: Executa os algoritmos utilizando apenas uma amostra do conjunto de dados.
  - **`False`**: Executa os algoritmos utilizando o conjunto de dados completo.
  
- **`SAMPLE_SIZE = 1.0` até `0.01`**: Define o **tamanho da amostra** a ser gerada, representando uma fração do total de dados.
  - **Exemplo**: `SAMPLE_SIZE = 0.1` corresponde a 10% do conjunto original.
  - Essa constante é utilizada para determinar a quantidade de dados a ser extraída quando `USE_SAMPLE` é configurado como `True`.


In [ ]:
USE_SAMPLE = True
SAMPLE_SIZE = 0.1

### Conversão de Arquivo `.txt.gz` para `.csv`
Este trecho do código realiza a conversão de um arquivo compactado (`.txt.gz`) contendo pares de vértices (arestas de um grafo) em um arquivo `.csv` com cabeçalhos apropriados.

- **Verificação de Existência do Arquivo de Entrada**: O código verifica se o arquivo de entrada, especificado pela variável `input_path`, existe. Se o arquivo não for encontrado, uma mensagem de erro é exibida, e o programa é encerrado com o código de erro 1.
  
- **Verificação de Existência do Diretório de Destino**: O código também verifica se o diretório de destino, especificado pela variável `output_path`, existe. Caso contrário, ele cria o diretório necessário para armazenar o arquivo `.csv` de saída.

- **Abertura dos Arquivos**: O arquivo compactado `.txt.gz` é aberto no modo texto (`'rt'`), enquanto o arquivo `.csv` é aberto no modo escrita (`'w'`), permitindo a gravação das arestas extraídas.

- **Escrita do Cabeçalho**: O cabeçalho `['source', 'target']` é escrito no arquivo `.csv`. Esses cabeçalhos correspondem aos vértices de cada aresta do grafo, sendo que cada par de vértices representará uma aresta.

- **Processamento das Linhas do Arquivo de Entrada**: Para cada linha no arquivo `.txt.gz`, o código realiza os seguintes passos:
  - Divide a linha em dois elementos separados por espaço em branco.
  - Verifica se a linha contém exatamente dois elementos (representando uma aresta). Se sim, escreve esse par de vértices no arquivo `.csv`.

Esse processo é útil para converter dados de grafos armazenados de forma compactada em um formato estruturado e mais facilmente utilizável, como o `.csv`.


In [ ]:
if not(os.path.exists(input_path)):
    print(f"Arquivo {input_path} não encontrado.")
    exit(1)

if not os.path.exists(os.path.dirname(output_path)):
    print(f"Criando diretório {os.path.dirname(output_path)}")
    os.makedirs(os.path.dirname(output_path))


with gzip.open(input_path, 'rt') as infile, open(output_path, 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['source', 'target'])  
    for line in infile:
        nodes = line.strip().split()
        if len(nodes) == 2:
            writer.writerow(nodes)

print(f"Arquivo CSV salvo como {output_path}")

### Geração e Salvamento de Amostra do Arquivo CSV

Este trecho de código realiza a geração de uma amostra aleatória de um arquivo CSV e a salva em um novo arquivo. Abaixo estão os detalhes de cada etapa do processo.

- **Exibição da Mensagem de Início**: O código começa exibindo uma mensagem que informa que uma amostra do arquivo CSV está sendo gerada, especificando o tamanho da amostra com base na constante `SAMPLE_SIZE`.

- **Leitura do Arquivo CSV**: O arquivo CSV completo é lido e carregado em um DataFrame (`df`) utilizando a função `pd.read_csv()`. O arquivo CSV está localizado no caminho especificado pela variável `output_path`.

- **Geração da Amostra Aleatória**: O código utiliza o método `sample()` do Pandas para extrair uma amostra aleatória do DataFrame. O parâmetro `frac=SAMPLE_SIZE` define a fração dos dados a ser extraída, com base no valor de `SAMPLE_SIZE`. O parâmetro `random_state=1` garante que a amostra seja reprodutível, ou seja, a mesma amostra será gerada toda vez que o código for executado com as mesmas configurações.

- **Salvamento da Amostra em um Novo Arquivo CSV**: Após a geração da amostra, ela é salva em um novo arquivo CSV, cujo caminho é especificado pela variável `CSV_FILE_SAMPLE`. O parâmetro `index=False` garante que os índices do DataFrame não sejam incluídos no arquivo CSV gerado.

- **Exibição da Mensagem de Conclusão**: Por fim, uma mensagem é exibida para informar que a amostra foi salva com sucesso no novo arquivo, cujo caminho é fornecido pela variável `CSV_FILE_SAMPLE`.

Este processo é útil para trabalhar com uma porção representativa dos dados, facilitando a análise e o desenvolvimento sem precisar carregar ou processar o conjunto completo de dados.


In [ ]:
print(f"Gerando uma Amostra do arquivo CSV de tamanho {SAMPLE_SIZE}")
#Lê o arquivo CSV e gera uma amostra
df = pd.read_csv(output_path)
sample_df = df.sample(frac=SAMPLE_SIZE, random_state=1)  # Amostra aleatória
sample_df.to_csv(CSV_FILE_SAMPLE, index=False)
print(f"Amostra salva como {CSV_FILE_SAMPLE}")


###  Verificação da Existência dos Arquivos `.csv`

Este trecho de código realiza a verificação da existência dos arquivos `.csv` (completo e amostra), e define qual será utilizado durante a execução com base na constante `USE_SAMPLE`.

- **Seleção do Arquivo Base**: O código define a variável `CURRENT_CSV_FILE` com base no valor de `USE_SAMPLE`:
  - Se `USE_SAMPLE = True`, o arquivo de amostra (`CSV_FILE_SAMPLE`) será usado.
  - Caso contrário, o arquivo completo (`CSV_FILE`) será utilizado.

- **Verificação do Arquivo de Amostra**:
  - Exibe uma mensagem indicando que está verificando a existência do arquivo de amostra.
  - Informa se o arquivo de amostra existe ou não.
  - Se não existir, informa que é necessário gerar um novo arquivo de amostra, que pode ser Gerado em [Geração de Amostra](#geração-e-salvamento-de-amostra-do-arquivo-csv).

- **Verificação do Arquivo Completo**:
  - Verifica se o arquivo `.csv` completo existe.
  - Exibe uma mensagem indicando o resultado dessa verificação.
  - Se o arquivo não existir, informa que é necessário gerar um novo arquivo completo, que pode ser gerado em [Geração de Arquivo Completo](#-conversão-de-arquivo-txtgz-para-csv).

- **Validação Final e Controle de Erro**:
  - Verifica se o arquivo selecionado (`CURRENT_CSV_FILE`) realmente existe.
  - Caso não exista, exibe uma mensagem de erro e encerra a execução com `exit()`.

- **Mensagem Final**:
  - Exibe qual arquivo será utilizado na execução do restante do código, seja o completo ou a amostra.

Esse processo garante que o ambiente esteja corretamente configurado, evitando erros causados por arquivos ausentes ou caminhos incorretos.


In [ ]:
if USE_SAMPLE:
    CURRENT_CSV_FILE = CSV_FILE_SAMPLE
else:
    CURRENT_CSV_FILE = CSV_FILE

if os.path.exists(CSV_FILE_SAMPLE):
    print("O arquivo de amostra existe.")
else:
    print("O arquivo de amostra não existe. Gere um novo arquivo de amostra.")

if os.path.exists(CSV_FILE):
    print("O arquivo completo existe.")
else:
    print("O arquivo completo não existe. Gere um novo arquivo completo.")


if not os.path.exists(CURRENT_CSV_FILE):
    print(f"ERRO: O arquivo {CURRENT_CSV_FILE} não foi encontrado. Verifique o caminho.")
    exit()

print(f"\n--- Usando o arquivo: {CURRENT_CSV_FILE} ---")

### TODO

In [ ]:
# --- 1. Coletar Dados e 2. Construir Grafo ---
print("\n1. Lendo dados e construindo o grafo...")
# Load dataset (assumindo formato: source, target para cada interação)
df = pd.read_csv(CURRENT_CSV_FILE)

if 'source' not in df.columns or 'target' not in df.columns:
    print("ERRO: O CSV deve conter colunas nomeadas 'source' e 'target'.")
    # Se suas colunas tiverem outros nomes, ajuste aqui ou no arquivo:
    # Exemplo: se as colunas forem as duas primeiras sem nome
    if len(df.columns) >= 2:
        print(f"Assumindo que as duas primeiras colunas são source e target: {df.columns[0]}, {df.columns[1]}")
        df.columns = ['source', 'target'] + list(df.columns[2:])
    else:
        print("Não foi possível identificar as colunas source e target.")
        exit()


g = nx.DiGraph()

# Adicionar arestas com peso baseado na frequência de interação
for _, row in df.iterrows():
    u = row['source']
    v = row['target']
    if g.has_edge(u, v):
        g[u][v]['weight'] += 1
    else:
        g.add_edge(u, v, weight=1)

print(f"Grafo construído: Nós = {g.number_of_nodes()}, Arestas = {g.number_of_edges()}")

### TODO

In [ ]:
# 3.1 Algoritmo PageRank: Identificar nós mais influentes
print("\n3.1 Calculando PageRank...")
try:
    pagerank = nx.pagerank(g, alpha=0.85, weight='weight') # Usar 'weight' pode ser interessante
    # pagerank = nx.pagerank(g, alpha=0.85) # Ou sem peso, se preferir
    top_n_pagerank = 10
    top_pagerank_nodes = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:top_n_pagerank]
    print(f"Top {top_n_pagerank} nós mais influentes (PageRank):")
    for i, (node, score) in enumerate(top_pagerank_nodes, 1):
        print(f"{i}. Nó: {node} - PageRank Score: {score:.6f}")
except Exception as e:
    print(f"Erro ao calcular PageRank: {e}. Pode ser um grafo desconexo ou muito pequeno.")

### TODO

In [ ]:
print("\n3.2 Detectando comunidades (Label Propagation)...")
if g.number_of_nodes() > 0:
    g_undirected = None
    if nx.is_directed(g):
        print("Aviso: Os algoritmos Louvain e Girvan-Newman em NetworkX esperam grafos não direcionados.")
        print("Convertendo o grafo para não direcionado para aplicar os algoritmos.")
        g_undirected = g.to_undirected()
    else:
        print("O grafo já é não direcionado. Usando-o diretamente.")
        g_undirected = g # Ou g.copy() se quiser evitar modificações acidentais no original

    # --- 3.3.1 Algoritmo de Louvain ---
    print("\n--- 3.3.1 Algoritmo de Louvain ---")
    try:
        # O algoritmo de Louvain é heurístico e pode dar resultados ligeiramente diferentes.
        # O parâmetro 'seed' pode ser usado para reprodutibilidade.
        # O parâmetro 'resolution' ajusta a granularidade das comunidades.
        communities_louvain_sets = louvain_communities(g_undirected, seed=42, resolution=1.0)
        communities_louvain = [sorted(list(c)) for c in communities_louvain_sets] # Converter para lista de listas e ordenar

        print(f"Número de comunidades encontradas (Louvain): {len(communities_louvain)}")
        if communities_louvain:
            print("Tamanhos das 5 maiores comunidades (Louvain):")
            sorted_communities_louvain = sorted(communities_louvain, key=len, reverse=True)
            for i, comm in enumerate(sorted_communities_louvain[:5]):
                example_members = str(comm[:3]).strip('[]')
                print(f"Comunidade {i+1}: {len(comm)} membros. Ex: {example_members}...")
        else:
            print("Nenhuma comunidade detectada pelo algoritmo de Louvain.")
    except Exception as e:
        print(f"Erro ao detectar comunidades com Louvain: {e}")
        communities_louvain = []

    # --- 3.3.2 Algoritmo de Girvan-Newman ---
    print("\n--- 3.3.2 Algoritmo de Girvan-Newman ---")
    try:
        # Girvan-Newman é computacionalmente mais intensivo.
        # Retorna um iterador de tuplas de frozensets. Cada tupla é uma partição.
        # O iterador produz partições em diferentes níveis de granularidade,
        # começando com todos os nós em uma comunidade e dividindo-as.
        # Para uma única partição, geralmente pegamos o primeiro ou um dos primeiros resultados.
        # Ou iteramos até um número desejado de comunidades.

        print("Executando Girvan-Newman (pode ser lento para grafos grandes)...")
        gn_communities_generator = girvan_newman(g_undirected)

        # Opção 1: Pegar o primeiro nível de partição (geralmente o mais grosseiro após a primeira divisão)
        # ou um nível específico. Para demonstração, pegaremos uma partição com um número
        # razoável de comunidades, se possível, ou a primeira.
        # Vamos tentar pegar a partição que resulta em um número de comunidades entre 2 e N/2 (aproximadamente)
        # ou simplesmente um número fixo de iterações.

        # Para este exemplo, vamos pegar a partição após algumas iterações (ex: 2 iterações para ter 3 comunidades, se possível)
        # Se o grafo for pequeno, pode haver menos iterações.
        # k_communities_target = 3 # Número desejado de comunidades (k)
        # Descomente a linha abaixo se quiser um número específico de comunidades (k)
        # e comente a linha `limited_gn_iterations = list(islice(gn_communities_generator, k_communities_target -1))`
        # communities_gn_at_k = None
        # for i, communities_tuple in enumerate(gn_communities_generator):
        #     if i == k_communities_target - 2: # Iteramos k-1 vezes para obter k comunidades (0-indexed)
        #         communities_gn_at_k = tuple(sorted(list(c)) for c in communities_tuple)
        #         break
        # if communities_gn_at_k is None: # Se não atingiu k, pega a última partição disponível
        #     print(f"Não foi possível obter exatamente {k_communities_target} comunidades. Usando a última partição gerada.")
        #     # Precisamos re-executar ou armazenar a última
        #     gn_communities_generator = girvan_newman(g_undirected) # Re-executa
        #     final_communities_tuple = None
        #     for comm_tuple in gn_communities_generator:
        #         final_communities_tuple = comm_tuple # Pega a última
        #     if final_communities_tuple:
        #       communities_gn_at_k = tuple(sorted(list(c)) for c in final_communities_tuple)

        # Abordagem mais simples: pegar o resultado após um número fixo de "cortes"
        # Ou a primeira partição não trivial
        num_cuts_for_gn = 2 # Número de "cortes" de arestas a serem feitos. Isso resultará em num_cuts_for_gn + 1 comunidades, se possível.
        
        # Tentativa de obter um número específico de comunidades
        # Se o grafo for muito pequeno, pode não ser possível obter 'num_cuts_for_gn + 1' comunidades distintas.
        desired_num_communities_gn = 3 # Por exemplo, tentamos obter 3 comunidades
        found_communities_gn = None

        for i, comm_level in enumerate(islice(gn_communities_generator, 10)): # Limita a 10 níveis para não demorar demais
            current_communities = tuple(sorted(list(c)) for c in comm_level)
            if len(current_communities) >= desired_num_communities_gn:
                found_communities_gn = current_communities
                print(f"Girvan-Newman: Partição encontrada com {len(found_communities_gn)} comunidades (nível {i+1}).")
                break
            # Guardar o último caso encontremos menos que o desejado
            found_communities_gn = current_communities 
        
        if not found_communities_gn: # Se o gerador estiver vazio (grafo muito pequeno, ex: 1 nó)
            # Pegar a primeira partição (geralmente todos os nós em uma comunidade, ou logo após a primeira divisão)
            # Re-inicializar o gerador se ele foi consumido
            gn_communities_generator = girvan_newman(g_undirected)
            try:
                first_level_communities_gn = next(gn_communities_generator)
                # Converter para lista de listas e ordenar
                found_communities_gn = [sorted(list(c)) for c in first_level_communities_gn]
                print(f"Girvan-Newman: Usando o primeiro nível de partição com {len(found_communities_gn)} comunidades.")
            except StopIteration:
                print("Girvan-Newman: Não foi possível gerar partições (grafo pode ser muito pequeno ou já desconectado).")
                found_communities_gn = []

        communities_gn = found_communities_gn

        if communities_gn:
            print(f"Número de comunidades encontradas (Girvan-Newman): {len(communities_gn)}")
            print("Tamanhos das 5 maiores comunidades (Girvan-Newman):")
            # communities_gn já é uma tupla de listas ordenadas (ou lista de listas)
            sorted_communities_gn = sorted(communities_gn, key=len, reverse=True)
            for i, comm in enumerate(sorted_communities_gn[:5]):
                example_members = str(comm[:3]).strip('[]') # comm já é uma lista
                print(f"Comunidade {i+1}: {len(comm)} membros. Ex: {example_members}...")
        else:
            print("Nenhuma comunidade detectada pelo algoritmo de Girvan-Newman com os critérios atuais.")

    except Exception as e:
        print(f"Erro ao detectar comunidades com Girvan-Newman: {e}")
        communities_gn = []

else:
    print("Grafo vazio, pulando detecção de comunidades.")
    communities_louvain = []
    communities_gn = []


### TODO

In [ ]:
print("\n3.3 Calculando medidas de centralidade...")
top_n_centrality = 5

if g.number_of_nodes() > 0:
    # Grau de centralidade (Out-Degree)
    raw_out_degrees = {node: val for node, val in g.out_degree(weight='weight')}
    top_out_degree = sorted(raw_out_degrees.items(), key=lambda x: x[1], reverse=True)[:top_n_centrality]
    print(f"\nTop {top_n_centrality} nós por Centralidade de Saída (Out-Degree):")
    for i, (node, score) in enumerate(top_out_degree, 1):
        print(f"{i}. Nó: {node} - Out-Degree: {score}")

    # Conexão de centralidade (Betweenness Centrality)
    print(f"\nCalculando Centralidade de Intermediação (Betweenness)... (Pode demorar)")
    try:
        betweenness_centrality = nx.betweenness_centrality(g, weight='weight', normalized=True)
        top_betweenness = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:top_n_centrality]
        print(f"Top {top_n_centrality} nós por Centralidade de Intermediação:")
        for i, (node, score) in enumerate(top_betweenness, 1):
            print(f"{i}. Nó: {node} - Betweenness Score: {score:.6f}")
    except Exception as e:
        print(f"Erro ao calcular Betweenness Centrality: {e}")

    # Proximidade de centralidade (Closeness Centrality)
    print(f"\nCalculando Centralidade de Proximidade (Closeness)... (Pode demorar)")
    largest_scc = None
    if not nx.is_strongly_connected(g):
        print("O grafo não é fortemente conectado. Calculando Closeness para o maior componente fortemente conectado.")
        scc_nodes_list = list(nx.strongly_connected_components(g))
        if scc_nodes_list: # Verifica se a lista não está vazia
            largest_scc_nodes = max(scc_nodes_list, key=len, default=None)
            if largest_scc_nodes and len(largest_scc_nodes) > 1:
                 largest_scc = g.subgraph(largest_scc_nodes)
            else:
                print("Não foi possível encontrar um componente fortemente conectado adequado para Closeness.")
        else:
            print("Nenhum componente fortemente conectado encontrado.")
    else:
        largest_scc = g

    if largest_scc and largest_scc.number_of_nodes() > 1:
        try:
            closeness_centrality = nx.closeness_centrality(largest_scc)
            top_closeness = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:top_n_centrality]
            print(f"Top {top_n_centrality} nós por Centralidade de Proximidade (no maior SCC):")
            for i, (node, score) in enumerate(top_closeness, 1):
                print(f"{i}. Nó: {node} - Closeness Score: {score:.6f}")
        except Exception as e:
            print(f"Erro ao calcular Closeness Centrality: {e}")
    else:
        print("Não foi possível calcular Closeness Centrality (sem componente adequado ou grafo muito pequeno).")
else:
    print("Grafo vazio, pulando cálculo de centralidades.")
print("\n4. Visualização e Exportação...")
if g.number_of_nodes() > 0 and g.number_of_nodes() < 500:
    print("Tentando desenhar o grafo com Matplotlib...")
    plt.figure(figsize=(15, 10))
    pos = nx.spring_layout(g, k=0.15, iterations=20)

    # Colorir nós por comunidade (Label Propagation)
    node_colors_lp = ['gray'] * g.number_of_nodes()
    if communities_lp: # Usar as comunidades do Label Propagation
        node_to_community_id_lp = {}
        for i, comm_nodes in enumerate(communities_lp):
            for node in comm_nodes:
                node_to_community_id_lp[node] = i
        
        # Criar lista de cores na ordem de g.nodes()
        # Usar um colormap diferente para distinguir visualmente
        color_map_lp = plt.cm.get_cmap('tab20', len(communities_lp)) # 'tab20' tem mais cores distintas
        
        # Mapear cores para os nós na ordem correta que NetworkX os desenhará.
        # A forma mais segura é iterar sobre g.nodes() para criar a lista de cores.
        final_node_colors_lp = []
        node_list_for_drawing = list(g.nodes()) # Obter a ordem dos nós como NetworkX pode usá-los
        for node in node_list_for_drawing:
            community_id = node_to_community_id_lp.get(node)
            if community_id is not None:
                final_node_colors_lp.append(color_map_lp(community_id))
            else:
                final_node_colors_lp.append('lightgray') # Cor para nós sem comunidade atribuída

        nx.draw(g, pos, with_labels=False, node_color=final_node_colors_lp, node_size=15, width=0.1, alpha=0.7, arrows=True)
    else: # Se não houver comunidades, desenhar sem cores específicas de comunidade
        nx.draw(g, pos, with_labels=False, node_size=15, width=0.1, alpha=0.7, arrows=True)

    plt.title(f"Visualização do Grafo Direcionado (Nós: {g.number_of_nodes()}, Arestas: {g.number_of_edges()}) - Colorido por Label Propagation")
    plt.show()
else:
    print("Grafo muito grande para desenhar com Matplotlib ou vazio. Considere usar Gephi.")




### TODO

In [ ]:
gephi_output_file = "twitter_network_directed.gexf"
# Adicionar atributos de comunidade ao grafo para exportação, se desejar
if communities_lp:
    for i, comm_nodes in enumerate(communities_lp):
        for node in comm_nodes:
            if g.has_node(node): # Verificar se o nó ainda existe (pode não ser necessário aqui)
                g.nodes[node]['community_label_prop'] = i
try:
    nx.write_gexf(g, gephi_output_file)
    print(f"\nGrafo exportado para '{gephi_output_file}' para visualização no Gephi.")
    print("No Gephi, você poderá usar o atributo 'community_label_prop' para colorir os nós.")
except Exception as e:
    print(f"Erro ao exportar para Gephi: {e}")

### TODO

In [ ]:
# --- 5. Mostrar resultados plotando graus de distribuição ---
print("\n5. Plotando distribuição de graus...")
if g.number_of_nodes() > 0:
    # Out-degree (quem compartilha)
    out_degrees = [g.out_degree(n, weight='weight') for n in g.nodes()]
    if not out_degrees:
        print("Não foi possível obter out-degrees para plotagem.")
    else:
        degree_counts_out = Counter(out_degrees)
        degrees_out, counts_out = zip(*degree_counts_out.items())

    # In-degree (quem é alvo/referenciado)
    in_degrees = [g.in_degree(n, weight='weight') for n in g.nodes()]
    if not in_degrees:
        print("Não foi possível obter in-degrees para plotagem.")
    else:
        degree_counts_in = Counter(in_degrees)
        degrees_in, counts_in = zip(*degree_counts_in.items())


    plt.figure(figsize=(12, 12))

    # Plot Out-Degree Linear
    plt.subplot(2, 2, 1)
    plt.bar(degrees_out, counts_out, width=0.80, color='blue')
    plt.title("Distribuição de Out-Degree (Escala Linear)")
    plt.ylabel("Contagem")
    plt.xlabel("Out-Degree")

    # Plot Out-Degree Log-Log
    plt.subplot(2, 2, 2)
    plt.loglog(degrees_out, counts_out, marker='.', linestyle='none', color='blue')
    plt.title("Distribuição de Out-Degree (Escala Log-Log)")
    plt.ylabel("Contagem (Log)")
    plt.xlabel("Out-Degree (Log)")

    # Plot In-Degree Linear
    plt.subplot(2, 2, 3)
    plt.bar(degrees_in, counts_in, width=0.80, color='green')
    plt.title("Distribuição de In-Degree (Escala Linear)")
    plt.ylabel("Contagem")
    plt.xlabel("In-Degree")

    # Plot In-Degree Log-Log
    plt.subplot(2, 2, 4)
    plt.loglog(degrees_in, counts_in, marker='.', linestyle='none', color='green')
    plt.title("Distribuição de In-Degree (Escala Log-Log)")
    plt.ylabel("Contagem (Log)")
    plt.xlabel("In-Degree (Log)")

    plt.tight_layout()
    plt.show()
else:
    print("Grafo vazio, pulando plotagem de distribuição de graus.")

print("\n--- Análise Concluída ---")